In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# pd.options.plotting.backend = "plotly"

# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.tree import export_graphviz
from graphviz import Source

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
# from lightgbm import LGBMClassifier, plot_importance

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.ensemble import VotingClassifier, RandomForestClassifier

import time

# import pandas_profiling

In [5]:
pokemon = pd.read_csv('../0_data/pokemon_df_diff.csv')
# pokemon.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace= True)

### X, y split

In [6]:
cat_cols = ['#_x', 'First_pokemon', 'Second_pokemon', 
            'Legendary_first', 'Legendary_second']
conti_cols = ['HP_first', 'Attack_first',
              'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
              'Generation_first', 
              'HP_second', 'Attack_second', 'Defense_second',
              'Sp. Atk_second', 'Sp. Def_second', 'Speed_second', 'Generation_second']

y = ['Winner_mark']

In [7]:
X = pokemon[cat_cols + conti_cols]
y = pokemon[y]

In [8]:
assert len(pokemon.columns) == len (set(pokemon.columns))

In [9]:
# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=1)

# train / validation split
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=1)

y_train.shape, y_train.shape, X_test.shape, y_test.shape

((37500, 1), (37500, 1), (12500, 19), (12500, 1))

In [10]:
y_train = np.ravel (y_train, order = "c")
y_train.shape

(37500,)

In [11]:
assert len(X_train.columns) ==len(set(X_train.columns))

### Xgboost

In [12]:
pipeline_xgb = make_pipeline(StandardScaler(), XGBClassifier(random_state=1,objective='binary:logistic',),verbose=True)
pipeline_xgb.steps 

[('standardscaler', StandardScaler()),
 ('xgbclassifier',
  XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
                colsample_bynode=None, colsample_bytree=None, gamma=None,
                gpu_id=None, importance_type='gain', interaction_constraints=None,
                learning_rate=None, max_delta_step=None, max_depth=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=None, num_parallel_tree=None,
                random_state=1, reg_alpha=None, reg_lambda=None,
                scale_pos_weight=None, subsample=None, tree_method=None,
                validate_parameters=None, verbosity=None))]

In [13]:
%%time
pipeline_xgb.fit(X_train, y_train)

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[13:16:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[Pipeline] ..... (step 2 of 2) Processing xgbclassifier, total=   2.0s
CPU times: user 12.7 s, sys: 596 ms, total: 13.3 s
Wall time: 2.01 s


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1, random_state=1,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))],
         verbose=True)

In [14]:
accuracy_score(y_train, pipeline_xgb.predict(X_train)), accuracy_score(y_test, pipeline_xgb.predict(X_test))

(0.97376, 0.94984)